--- Day 11: Cosmic Expansion ---
You continue following signs for "Hot Springs" and eventually come across an observatory. The Elf within turns out to be a researcher studying cosmic expansion using the giant telescope here.

He doesn't know anything about the missing machine parts; he's only visiting for this research project. However, he confirms that the hot springs are the next-closest area likely to have people; he'll even take you straight there once he's done with today's observation analysis.

Maybe you can help him with the analysis to speed things up?

The researcher has collected a bunch of data and compiled the data into a single giant image (your puzzle input). The image includes empty space (.) and galaxies (#). For example:

...#......
.......#..
#.........
..........
......#...
.#........
.........#
..........
.......#..
#...#.....
The researcher is trying to figure out the sum of the lengths of the shortest path between every pair of galaxies. However, there's a catch: the universe expanded in the time it took the light from those galaxies to reach the observatory.

Due to something involving gravitational effects, only some space expands. In fact, the result is that any rows or columns that contain no galaxies should all actually be twice as big.

In the above example, three columns and two rows contain no galaxies:

   v  v  v
 ...#......
 .......#..
 #.........
>..........<
 ......#...
 .#........
 .........#
>..........<
 .......#..
 #...#.....
   ^  ^  ^
These rows and columns need to be twice as big; the result of cosmic expansion therefore looks like this:

....#........
.........#...
#............
.............
.............
........#....
.#...........
............#
.............
.............
.........#...
#....#.......
Equipped with this expanded universe, the shortest path between every pair of galaxies can be found. It can help to assign every galaxy a unique number:

....1........
.........2...
3............
.............
.............
........4....
.5...........
............6
.............
.............
.........7...
8....9.......
In these 9 galaxies, there are 36 pairs. Only count each pair once; order within the pair doesn't matter. For each pair, find any shortest path between the two galaxies using only steps that move up, down, left, or right exactly one . or # at a time. (The shortest path between two galaxies is allowed to pass through another galaxy.)

For example, here is one of the shortest paths between galaxies 5 and 9:

....1........
.........2...
3............
.............
.............
........4....
.5...........
.##.........6
..##.........
...##........
....##...7...
8....9.......
This path has length 9 because it takes a minimum of nine steps to get from galaxy 5 to galaxy 9 (the eight locations marked # plus the step onto galaxy 9 itself). Here are some other example shortest path lengths:

Between galaxy 1 and galaxy 7: 15
Between galaxy 3 and galaxy 6: 17
Between galaxy 8 and galaxy 9: 5
In this example, after expanding the universe, the sum of the shortest path between all 36 pairs of galaxies is 374.

Expand the universe, then find the length of the shortest path between every pair of galaxies. What is the sum of these lengths?

In [124]:
from dataclasses import dataclass
from __future__ import annotations

import numpy as np



@dataclass
class GalaxyMap:
    galaxy_input_array: np.ndarray

    @classmethod
    def from_str(cls, input_str: str) -> GalaxyMap:
        map_list_of_rows = input_str.strip().split("\n")
        n_rows_input = len(map_list_of_rows)
        n_columns_input = len(map_list_of_rows[0])
        galaxy_input_array = np.zeros(shape=(n_rows_input, n_columns_input))
        for row_i, map_row in enumerate(map_list_of_rows):
            for col_i, map_char in enumerate(map_row):
                if map_char=="#":
                    galaxy_input_array[row_i, col_i] = 1

        return cls(
            galaxy_input_array=galaxy_input_array,
            )


    @property
    def adjusted_galaxy_map_array(self) -> np.ndarray:
        is_row_empty = self.galaxy_input_array.sum(axis=1) == 0
        is_col_empty = self.galaxy_input_array.sum(axis=0) == 0
        
        input_rows_n = self.galaxy_input_array.shape[0]
        input_cols_n = self.galaxy_input_array.shape[1]
        new_rows_n = input_rows_n + sum(is_row_empty)
        new_cols_n = input_cols_n + sum(is_col_empty)

        cum_rows_empty = is_row_empty.cumsum()
        cum_cols_empty = is_col_empty.cumsum()

        adjusted_array = np.zeros(shape=(new_rows_n, new_cols_n))

        for (input_row_i, input_col_i) in np.argwhere(self.galaxy_input_array):
            for input_col_i in range(input_cols_n):
                adj_row_i = input_row_i + cum_rows_empty[input_row_i]
                adj_col_i = input_col_i + cum_cols_empty[input_col_i]
                adjusted_array[adj_row_i, adj_col_i] = self.galaxy_input_array[input_row_i, input_col_i]

        return adjusted_array


    @property
    def galaxy_indices(self) -> np.ndarray:
        return np.argwhere(self.adjusted_galaxy_map_array)


    @property
    def sum_of_galaxy_pair_distances(self) -> int:
        running_sum = 0

        n_galaxy = len(self.galaxy_indices)
        for i in range(n_galaxy):
            for j in range(i+1, n_galaxy):
                running_sum += (
                    abs(self.galaxy_indices[i][0] - self.galaxy_indices[j][0])
                    + abs(self.galaxy_indices[i][1] - self.galaxy_indices[j][1])
                )
        return running_sum


In [125]:
test_input = """
...#......
.......#..
#.........
..........
......#...
.#........
.........#
..........
.......#..
#...#.....
"""



In [126]:
test_map = GalaxyMap.from_str(test_input)

In [127]:
test_map

GalaxyMap(galaxy_input_array=array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 1., 0., 0., 0., 0., 0.]]))

In [128]:
assert test_map.sum_of_galaxy_pair_distances == 374